In [6]:
import edward as ed
import tensorflow as tf

from edward.models import Bernoulli, Normal
from scipy.misc import imsave
from keras.layers import Dense

from tensorflow.contrib import slim
from tensorflow.examples.tutorials.mnist import input_data


Using TensorFlow backend.


In [7]:
M = 100 # batch size during training
d = 2 # latent variable dimension


In [11]:
# Probability model (subgraph)
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))
h = Dense(256, activation='relu')(z.value())
x = Bernoulli(logits=Dense(28 * 28, activation=None)(h))


In [36]:
z.value()

<tf.Tensor 'Normal_3/sample/Reshape:0' shape=(100, 2) dtype=float32>

In [37]:
# Variational model (subgraph)
x_ph = tf.placeholder(tf.int32, [M, 28 * 28])
qh = Dense(256, activation='relu')(tf.cast(x_ph, tf.float32))
qz = Normal(loc=Dense(d, activation=None)(qh),
scale=Dense(d, activation='softplus')(qh))


In [38]:
# Bind p(x, z) and q(z | x) to the same TensorFlow placeholder for x.
mnist = input_data.read_data_sets("data/mnist", one_hot=True)
data = {x: x_ph}


Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [50]:
inference = ed.KLqp({z: qz}, data)
optimizer = tf.train.RMSPropOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)



In [51]:
sess = ed.get_session()
tf.global_variables_initializer().run()

n_epoch = 100
n_iter_per_epoch = 1000
for _ in range(n_epoch):
    for _ in range(n_iter_per_epoch):
        x_train, _ = mnist.train.next_batch(M)
        info_dict = inference.update(feed_dict={x_ph: x_train})

        # Generate images.
        imgs = x.value().eval()
        for m in range(M):
            imsave("img/%d.png" % m, imgs[m].reshape(28, 28))

/home/willtai/4thyearproject/env_edward/lib/python3.5/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


KeyboardInterrupt: 